In [26]:
from typing import Final

from bayes_opt import BayesianOptimization
from bayes_opt.event import Events
from bayes_opt.logger import JSONLogger
from bayes_opt.util import load_logs
from numpy import where
from pandas import DataFrame, Series, concat, read_csv
from sklearn.metrics import f1_score
from sklearn.svm import OneClassSVM

MIN_SAMPLES: Final[int] = 0
LOGS_PATH: Final[str] = "../reports/logs_sequence.log"
logger = JSONLogger(path=LOGS_PATH, reset=False)

In [27]:
X_train = read_csv("../data/PAMAP2/x_train_data.csv")
X_test = read_csv("../data/PAMAP2/x_test_data.csv")
y_train = read_csv("../data/PAMAP2/y_train_data.csv")
y_test = read_csv("../data/PAMAP2/y_test_data.csv")

X_train["activity"] = y_train  # First 80% of the data
X_test["activity"] = y_test  # Last 20% of the data

# MIN_SAMPLES = X_train["activity"].value_counts().min()
MIN_SAMPLES = X_train["activity"].value_counts().sort_values().iloc[3]

models: dict[int, dict] = {}
training_data: DataFrame
testing_data: DataFrame
train_targets: Series
test_targets: Series

In [28]:
def objective_function(nu: float, gamma: float) -> float:
    """
    Objective function to optimize F1-Score on the test set.

    Args:
        nu (float): nu param to evaluate.
        gamma (float): gamma param to evaluate.

    Returns:
        float: F1-Score on the test set of this iteration.
    """
    oc_svm = OneClassSVM(kernel="rbf", nu=nu, gamma=gamma).fit(training_data)

    f1_train = f1_score(
        train_targets,
        where(oc_svm.predict(training_data) == 1, False, True),
        average="macro",
    )
    f1_test = f1_score(
        test_targets,
        where(oc_svm.predict(testing_data) == 1, False, True),
        average="macro",
    )
    print(f"\nF1 Score (Train): {f1_train}\nF1 Score (Test): {f1_test}")
    return float(f1_test)

In [29]:
for i in range(1, len((activities := X_train["activity"].unique()))):
    training = X_train[X_train["activity"].isin(activities[:i])].head(MIN_SAMPLES)
    testing = X_test[X_test["activity"].isin(activities[i : i + 1])].head(MIN_SAMPLES)

    training.loc[:, "isNovelty"], testing.loc[:, "isNovelty"] = False, True
    sampled_data = training.sample(n=int(0.2 * len(training)), random_state=42)
    novelty = concat([testing, sampled_data])

    training_data = training.drop(columns=["isNovelty"])
    train_targets = training["isNovelty"]
    # only current activity (as novelty)
    testing_data = novelty.drop(columns=["isNovelty"])
    test_targets = novelty["isNovelty"]

    optimizer = BayesianOptimization(
        f=objective_function,
        pbounds={"nu": (0.01, 0.5), "gamma": (1e-4, 1e-1)},
        random_state=42,
    )
    load_logs(optimizer, logs=[LOGS_PATH])
    print("New optimizer is now aware of {} points.".format(len(optimizer.space)))

    optimizer.subscribe(Events.OPTIMIZATION_STEP, logger)
    optimizer.maximize(init_points=5, n_iter=25)

    models[i] = optimizer.max  # type: ignore

New optimizer is now aware of 0 points.

F1 Score (Train): 0.34336147352264007
F1 Score (Test): 0.7795303711685515

F1 Score (Train): 0.4098903372646389
F1 Score (Test): 0.8666179163369635

F1 Score (Train): 0.4745762711864407
F1 Score (Test): 0.9608962619220839

F1 Score (Train): 0.36149253731343284
F1 Score (Test): 0.8015597031120053

F1 Score (Train): 0.39085860743272105
F1 Score (Test): 0.8417199044799836

F1 Score (Train): 0.46001893341748185
F1 Score (Test): 0.9355503786030874

F1 Score (Train): 0.49738588967867003
F1 Score (Test): 0.9969123413447807

F1 Score (Train): 0.49738588967867003
F1 Score (Test): 0.9969123413447807

F1 Score (Train): 0.49318801089918257
F1 Score (Test): 0.9895836939069856

F1 Score (Train): 0.4753495217071376
F1 Score (Test): 0.9615931855046034

F1 Score (Train): 0.4761846455246725
F1 Score (Test): 0.9634481719209901

F1 Score (Train): 0.49629106322854116
F1 Score (Test): 0.9933680482205001

F1 Score (Train): 0.4902591599642538
F1 Score (Test): 0.9839828